## Benchamrking a problems sets from pagmo2

- Define problems ~400k
- Solutions approaches:
    - Random solver:
        - Sobol
        - Random
    - Genetic solver:
        - nsga2
        - maco 
        - nspso
        - moead
    - Tutor Model
        - Portfolio. 1-3 versions
        - Nsga2 + ?
        - Score solution from solvers
        - Points per iteration
        - Rounds
            - How mach rounds do we need to reach quolity of genetic solvers?
            - How mach points do we need?

In [1]:
import datetime
import time
import sys
sys.path.append('..')

# --- Dependencies
import pygmo as pg
import numpy as np
import pandas as pd
# import dovpanda

import plotly
from plotly import graph_objs as go
from plotly import tools
import plotly.express as px

import sklearn.gaussian_process as gp
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

# from src.composite import PredictTutor, ModelsUnion
# from src.generator import SamplesGenerator
# from src.ploting import plot_mo

# from src.hypothesis.tpot_estimator import TpotWrp
# from src.hypothesis.custom_gp_kernel import KERNEL_MAUNA, KERNEL_SIMPLE, KERNEL_GPML

In [2]:
df = pd.read_csv("./data/scal_on_zdt_i11250.cxvq476avp.csv")
df

,problem_name,seed,problem_id,objectives,feature_dim,scal_method,loop_size,pop_ndf_x,pop_ndf_f,fevals,evolve_time,total_time,date,p_distance,hypervolume,ndf_space,ndf_size,error,final
0,ZDT1,42,1,2,2,bi,40,"[[0.9267079355992962, 1.3266113836488358e-07],...","[[0.9267079355992962, 0.037343912233401236], [...",160040.0,0.047559,0.047658,2020-02-18 20:10,0.002435,0.314486,0.071400,36.0,NaN,True
1,ZDT1,72,1,2,2,bi,40,"[[0.46283053107958233, 4.10382716427889e-05], ...","[[0.46283053107958233, 0.3199272271672657], [0...",160040.0,0.047049,0.047112,2020-02-18 20:10,0.001769,0.313385,0.075774,37.0,NaN,True
2,ZDT1,112,1,2,2,bi,40,"[[0.9445335740321337, 5.175916288363857e-06], ...","[[0.9445335740321337, 0.028152775532743], [0.2...",160040.0,0.047215,0.047262,2020-02-18 20:10,0.000730,0.314574,0.062963,39.0,NaN,True
3,ZDT1,774,1,2,2,bi,40,"[[0.009046799476173728, 2.082865735683287e-08]...","[[0.009046799476173728, 0.904885513657982], [0...",160040.0,0.046078,0.046124,2020-02-18 20:10,0.001174,0.310736,0.087714,36.0,NaN,True
4,ZDT1,111,1,2,2,bi,40,"[[0.4347756698675735, 0.001493382328159183], [...","[[0.4347756698675735, 0.34964887232181935], [0...",160040.0,0.051250,0.051306,2020-02-18 20:10,0.000733,0.312386,0.069745,38.0,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,zdt,42,6,10,10,weighted,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Loop: \nfunction: generic_ctor_impl\nwhere: /h...,False
11246,zdt,72,6,10,10,weighted,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Loop: \nfunction: generic_ctor_impl\nwhere: /h...,False
11247,zdt,112,6,10,10,weighted,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Loop: \nfunction: generic_ctor_impl\nwhere: /h...,False
11248,zdt,774,6,10,10,weighted,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Loop: \nfunction: generic_ctor_impl\nwhere: /h...,False


In [3]:
df = df.loc[(df['final'] >= True) & (df['problem_id'] == 4) & (df['objectives'] == 2) & (df['feature_dim'] == 2)]
df

,problem_name,seed,problem_id,objectives,feature_dim,scal_method,loop_size,pop_ndf_x,pop_ndf_f,fevals,evolve_time,total_time,date,p_distance,hypervolume,ndf_space,ndf_size,error,final
45,ZDT4,42,4,2,2,bi,40,"[[0.23789718202667678, -9.09508448320985e-06],...","[[0.23789718202667678, 0.5122530022741296], [0...",160040.0,0.045504,0.045548,2020-02-18 20:10,0.213645,0.374824,0.191411,22.0,NaN,True
46,ZDT4,72,4,2,2,bi,40,"[[0.9999997008260022, -0.49936746318474384], [...","[[0.9999997008260022, 0.1317913798563954], [0....",160040.0,0.047770,0.047802,2020-02-18 20:10,0.115733,0.351547,0.201344,20.0,NaN,True
47,ZDT4,112,4,2,2,bi,40,"[[0.7649278791567132, -0.49969675999700525], [...","[[0.7649278791567132, 0.27202536926585325], [0...",160040.0,0.045646,0.045689,2020-02-18 20:10,0.109481,0.339603,0.178700,19.0,NaN,True
48,ZDT4,774,4,2,2,bi,40,"[[0.016644801957787773, -0.49934445590592375],...","[[0.016644801957787773, 1.105459554448558], [0...",160040.0,0.046596,0.046640,2020-02-18 20:10,0.024111,0.278390,0.173042,11.0,NaN,True
49,ZDT4,111,4,2,2,bi,40,"[[0.7893194198707386, 1.8809280106219092e-06],...","[[0.7893194198707386, 0.11156349855006198], [0...",160040.0,0.046828,0.046875,2020-02-18 20:10,0.057206,0.326363,0.210568,15.0,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055,ZDT4,42,4,2,2,weighted,800,"[[0.9999968477576955, -1.3751733287103453e-05]...","[[0.9999968477576955, 1.6508747717853387e-06],...",3200800.0,0.940540,0.940911,2020-02-18 20:31,0.014625,0.316058,0.029018,404.0,NaN,True
9056,ZDT4,72,4,2,2,weighted,800,"[[0.6888558312114405, 0.0017357719154134088], ...","[[0.6888558312114405, 0.17142058789840844], [0...",3200800.0,0.937253,0.937624,2020-02-18 20:31,0.004549,0.313838,0.031291,331.0,NaN,True
9057,ZDT4,112,4,2,2,weighted,800,"[[0.08590567447305202, -0.0004604289552663232]...","[[0.08590567447305202, 0.7070463378915197], [0...",3200800.0,0.941460,0.941815,2020-02-18 20:32,0.001117,0.321934,0.027348,271.0,NaN,True
9058,ZDT4,774,4,2,2,weighted,800,"[[0.6244292689733919, -0.002918962227050681], ...","[[0.6244292689733919, 0.21387016352775962], [0...",3200800.0,0.936404,0.936762,2020-02-18 20:32,0.006427,0.320759,0.030286,329.0,NaN,True


In [4]:
df.loc[(df['loop_size'] == 200)]

,problem_name,seed,problem_id,objectives,feature_dim,scal_method,loop_size,pop_ndf_x,pop_ndf_f,fevals,evolve_time,total_time,date,p_distance,hypervolume,ndf_space,ndf_size,error,final
4545,ZDT4,42,4,2,2,bi,200,"[[0.7980026943976454, 0.011107563646943555], [...","[[0.7980026943976454, 0.16158542343135837], [0...",800200.0,0.235339,0.235545,2020-02-18 20:14,0.038174,0.357770,0.070329,82.0,NaN,True
4546,ZDT4,72,4,2,2,bi,200,"[[0.038194773886993605, 0.0013649195555711646]...","[[0.038194773886993605, 0.805894099231632], [0...",800200.0,0.233878,0.234062,2020-02-18 20:14,0.008140,0.323354,0.059587,99.0,NaN,True
4547,ZDT4,112,4,2,2,bi,200,"[[0.9640580334663007, 5.643926565264022e-06], ...","[[0.9640580334663007, 0.01813544300046487], [0...",800200.0,0.232852,0.233042,2020-02-18 20:14,0.035271,0.342579,0.083152,45.0,NaN,True
4548,ZDT4,774,4,2,2,bi,200,"[[0.6624732085794882, 3.3892003693197434e-05],...","[[0.6624732085794882, 0.1860759667381208], [0....",800200.0,0.233721,0.233924,2020-02-18 20:14,0.029931,0.349585,0.062396,89.0,NaN,True
4549,ZDT4,111,4,2,2,bi,200,"[[0.39483218838298667, -1.2603275325516754e-05...","[[0.39483218838298667, 0.3716433495605524], [0...",800200.0,0.231805,0.231997,2020-02-18 20:14,0.062532,0.331779,0.109302,41.0,NaN,True
4550,ZDT4,42,4,2,2,tchebycheff,200,"[[0.7393891155268033, -0.003961917161764639], ...","[[0.7393891155268033, 0.14721170930590577], [0...",800200.0,0.233699,0.233878,2020-02-18 20:14,0.012227,0.323612,0.057801,76.0,NaN,True
4551,ZDT4,72,4,2,2,tchebycheff,200,"[[0.17275248154567258, 2.310131566207491e-05],...","[[0.17275248154567258, 0.584365300155197], [0....",800200.0,0.231048,0.231236,2020-02-18 20:14,0.014463,0.311086,0.076448,74.0,NaN,True
4552,ZDT4,112,4,2,2,tchebycheff,200,"[[0.2896307176423329, 2.1246839156239877e-05],...","[[0.2896307176423329, 0.46182675942101253], [0...",800200.0,0.232141,0.232332,2020-02-18 20:14,0.012460,0.316478,0.064387,73.0,NaN,True
4553,ZDT4,774,4,2,2,tchebycheff,200,"[[0.20273602492511528, 4.051789713596047e-05],...","[[0.20273602492511528, 0.5497388321392727], [0...",800200.0,0.230244,0.230448,2020-02-18 20:14,0.010972,0.320022,0.055311,94.0,NaN,True
4554,ZDT4,111,4,2,2,tchebycheff,200,"[[0.29011864187860736, 3.0040873382839524e-05]...","[[0.29011864187860736, 0.46137389564153236], [...",800200.0,0.233702,0.233891,2020-02-18 20:14,0.008675,0.312319,0.069864,56.0,NaN,True
